In [5]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

In [2]:
mnist_dataset = datasets.fetch_openml('mnist_784')

random_state = 0


X = mnist_dataset.data
y = mnist_dataset.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [6]:
# Reduce dimension to 2 with PCA
pca = make_pipeline(StandardScaler(),
                    PCA(0.95))

# Use a nearest neighbor classifier to evaluate the methods
decision_tree_classifier = DecisionTreeClassifier(max_depth=10, min_samples_split=5)

dim_reduction_methods = [('PCA', pca)]
        

dataset_columns = np.array(['zero','one','two','three','four','five','six','seven','eight','nine'])


In [ ]:
for i, (name, model) in enumerate(dim_reduction_methods):
    # Fit the method's model
    model.fit(X_train, y_train)

    # Fit a decision tree classifier on the training set
    decision_tree_classifier.fit(model.transform(X_train), y_train)

    accuracy_on_tranining = accuracy_score(decision_tree_classifier.predict(model.fit_transform(X_train)), y_train)
    
    
    accuracy_on_testing = accuracy_score(decision_tree_classifier.predict(model.transform(X_test)), y_test)
    
    print('Training accuracy ', accuracy_on_tranining)
    print('Testing accuracy ',  accuracy_on_testing)
    
    print(classification_report(y_test, prediction, target_names = dataset_columns))


In [ ]:
print(classification_report(y_true,y_pred))